<img src="logo.png" height="200" width="900"> 

#  Сбор данных: грязная работа вашими руками 

Пришло время самостоятельно написать парсер! Мы будем собирать данные [о ценах на книги.](http://books.toscrape.com)

In [1]:
### ╰( ͡° ͜ʖ ͡° )つ▬▬ι═══════  bzzzzzzzzzz
# will the code be with you
    
# Подгрузите все необходимые для работы пакеты.
import numpy as np
import pandas as pd
import requests
import bs4

# Если ваш код будет ругаться, что нет пакета lxml, установите его 
# Для этого выполните в одной из ячеек команду !pip3 install lxml
from typing import List

# your code here


Прогуляйтесь на сайт http://books.toscrape.com/ и изучите его структуру.  


# 1. Сбор ссылок на книги

Напишите функцию `get_soup`, которая по ссылке возвращает html-разметку страницы в формате `bs4` 

In [2]:
def get_page_soup(url_link):
    responce = requests.get(url_link)
    if (responce.ok):
        html = responce.content
        soup = bs4.BeautifulSoup(html,'html.parser')
    else:
        print(responce)
        soup = None
    return soup

In [3]:
main_url = 'http://books.toscrape.com/catalogue/'
page_number = 5

soup = get_page_soup(main_url + f'page-{page_number}.html')
soup.find_all('a')[0].text

'Books to Scrape'

In [4]:
books = soup.findAll(lambda tag: tag.name == 'article' and tag.get('class') == ["product_pod"])
[link.h3.a.attrs['href'] for link in books]

['princess-jellyfish-2-in-1-omnibus-vol-01-princess-jellyfish-2-in-1-omnibus-1_920/index.html',
 'princess-between-worlds-wide-awake-princess-5_919/index.html',
 'pop-gun-war-volume-1-gift_918/index.html',
 'political-suicide-missteps-peccadilloes-bad-calls-backroom-hijinx-sordid-pasts-rotten-breaks-and-just-plain-dumb-mistakes-in-the-annals-of-american-politics_917/index.html',
 'patience_916/index.html',
 'outcast-vol-1-a-darkness-surrounds-him-outcast-1_915/index.html',
 'orange-the-complete-collection-1-orange-the-complete-collection-1_914/index.html',
 'online-marketing-for-busy-authors-a-step-by-step-guide_913/index.html',
 'on-a-midnight-clear_912/index.html',
 'obsidian-lux-1_911/index.html',
 'my-paris-kitchen-recipes-and-stories_910/index.html',
 'masks-and-shadows_909/index.html',
 'mama-tried-traditional-italian-cooking-for-the-screwed-crude-vegan-and-tattooed_908/index.html',
 'lumberjanes-vol-2-friendship-to-the-max-lumberjanes-5-8_907/index.html',
 'lumberjanes-vol-1-bew

Напишите функцию `get_books_links`, которая находит в html-разметке страницы ссылки на странички с отдельными книгами. 

In [5]:
def get_book_links(page_soup) -> List[str]:
    books = soup.findAll(lambda tag: tag.name == 'article' and tag.get('class') == ["product_pod"])
    return [link.h3.a.attrs['href'] for link in books]

С помощью цикла соберите в лист `book_links` первые 200 книг.

In [6]:
book_links = []
page_number = 1
while len(book_links) < 200:
    link = main_url + 'page-{}.html'.format(page_number)
    soup = get_page_soup(link)
    res = get_book_links(soup)
    book_links.extend(res)
    page_number += 1

In [7]:
book_links = sorted(book_links)

In [8]:
# проверка, что задание решено корректно
assert len(book_links) == 200

# 2. Сбор информации о книгах 

Напишите несколько небольших функций, которые собирают различные данные об одной книге, необходимые для ответов на вопросы ниже. Информацию о книге собирайте в виде словаря вида 

```
{ 'name': 'Преступление и наказание', 'rating': 1, 'description': 'ужасно депрессивная книга', ... }

```

In [9]:
def get_name(soup):
    describe = soup.find('div', attrs={'class':'col-sm-6 product_main'})
    result = describe.h1.text
    return result

def get_rating(soup):
    describe = soup.find('p', attrs={'class':'star-rating'})
    answer = describe.get('class')[1]
    if (answer == 'One'):
        return 1
    elif (answer == 'Two'):
        return 2
    elif (answer == 'Three'):
        return 3
    elif (answer == 'Four'):
        return 4
    elif (answer == 'Five'):
        return 5
    else:
        return 0

In [10]:
def get_descr(soup):
    describe = soup.findAll('p')[3]
    return describe.text

In [11]:
def get_tax(soup):
    describe = soup.findAll('td')[4]
    return float(describe.text[1:])

In [12]:
def get_price(soup):
    describe = soup.findAll('td')[2]
    return float(describe.text[1:])

In [13]:
def get_attr(link):
    result = {}
    main_link = main_url + link
    soup = get_page_soup(main_link)
    result['name'] = get_name(soup)
    result['rating'] = get_rating(soup)
    result['description'] = get_descr(soup).replace("\n", '')
    result['tax'] = get_tax(soup)
    result['price'] = get_price(soup)
    return result

In [14]:
get_attr("the-bridge-to-consciousness-im-writing-the-bridge-between-science-and-our-old-and-new-beliefs_840/index.html")

{'name': "The Bridge to Consciousness: I'm Writing the Bridge Between Science and Our Old and New Beliefs.",
 'rating': 3,
 'description': '',
 'tax': 0.0,
 'price': 32.0}

Пройдите циклом по всем сыслкам из списка `book_links` и соберите данные о книгах в вектор `book_info`. 

In [15]:
book_info = []
for link in book_links:
    book_info.append(get_attr(link))

Превратим вектор из информации в полноценную таблицу с данными. 

In [16]:
book_info_df = pd.DataFrame(book_info)
print(book_info_df.shape)
book_info_df.head()

(200, 5)


,name,rating,description,tax,price
0,A Court of Thorns and Roses (A Court of Thorns...,1,"A thrilling, seductive new series from New Yor...",0.0,52.37
1,A Fierce and Subtle Poison,4,Everyone knows the legends about the cursed gi...,0.0,28.13
2,A Flight of Arrows (The Pathfinders #2),5,October 1776--August 1777It is said that what ...,0.0,55.53
3,A Light in the Attic,3,It's hard to imagine a world without A Light i...,0.0,51.77
4,A Murder in Time,1,"Beautiful and brilliant, Kendra Donovan is a r...",0.0,16.64


In [17]:
book_info_df

,name,rating,description,tax,price
0,A Court of Thorns and Roses (A Court of Thorns...,1,"A thrilling, seductive new series from New Yor...",0.0,52.37
1,A Fierce and Subtle Poison,4,Everyone knows the legends about the cursed gi...,0.0,28.13
2,A Flight of Arrows (The Pathfinders #2),5,October 1776--August 1777It is said that what ...,0.0,55.53
3,A Light in the Attic,3,It's hard to imagine a world without A Light i...,0.0,51.77
4,A Murder in Time,1,"Beautiful and brilliant, Kendra Donovan is a r...",0.0,16.64
...,...,...,...,...,...
195,"William Shakespeare's Star Wars: Verily, A New...",4,MAY THE VERSE BE WITH YOU!Return once more to ...,0.0,43.30
196,Without Borders (Wanderlove #1),2,"For Annie London, a month in a Central America...",0.0,45.07
197,Worlds Elsewhere: Journeys Around Shakespeare’...,5,"Anti-apartheid activist, Bollywood screenwrite...",0.0,40.30
198,You Are What You Love: The Spiritual Power of ...,4,You are what you love. But you might not love ...,0.0,21.87


Теперь, когда все данные собраны, настало время ответить на несколько вопросов:

- У скольких книг отсутствует описание? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans1`. 

In [18]:
ans1 = book_info_df[book_info_df["description"] == ''].shape[0]

In [19]:
print(ans1)

1


In [20]:
# проверка, что задание решено корректно

- Сколько раз в данных встречается налог, больший нуля?  Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans2`. 

In [21]:
ans2 = book_info_df[book_info_df["tax"] > 0].shape[0]

In [22]:
print(ans2)

0


In [23]:
# проверка, что задание решено корректно

- Сколько раз рейтинг книги составлял пять звезд? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans3`. 

In [24]:
ans3 = book_info_df[book_info_df["rating"] == 5].shape[0]

In [25]:
print(ans3)

37


In [26]:
# проверка, что задание решено корректно


- Какова средняя цена книг (без учета налога)? Положите число, получившееся в результате ваших манипуляций с таблицей, в переменную `ans4`. 

In [27]:
ans4 = book_info_df["price"].mean()

In [28]:
print(ans4)

34.79625000000001


In [29]:
# проверка, что задание решено корректно
